# Exploring the German-Korean Tatoeba dataset
[Github](https://github.com/Helsinki-NLP/Tatoeba-Challenge)

[Huggingface](https://huggingface.co/datasets/Helsinki-NLP/tatoeba_mt)

In [1]:
from tatoeba import preprocess, analysis
from rude_nmt import label_german
import re

In [2]:
subtitle_data = preprocess.get_subtitle_dataset(force_renew=True)

Using custom data configuration data-ddc4346cf51291d1
Found cached dataset csv (/Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-916b621a1790f10d_00000_of_00008.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-916b621a1790f10d_00001_of_00008.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-916b621a1790f10d_00002_of_00008.arrow


...

Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-49f5bd6f10f6e4fd.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-51f4ed4b4bde5dc8.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5a2a09afaf6eb181.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-15181fceede7d700.arrow


Loading cached processed dataset at /Users/ludwig/.cache/huggingface/datasets/csv/data-ddc4346cf51291d1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c9af3658e1e52562.arrow


In [3]:
len(subtitle_data)
print(subtitle_data)


Dataset({
    features: ['id', 'source', 'target'],
    num_rows: 996525
})


In [4]:
subtitle_data = subtitle_data.map(label_german.annotate_formality)

  0%|          | 0/996525 [00:00<?, ?ex/s]

In [5]:
subtitle_data.to_csv("../data/subtitle_data.csv", num_proc=8)

Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

160097377

In [ ]:
one_word_trg = analysis.get_one_word_sentences(subtitle_data)
print(one_word_trg)

In [6]:
dat = subtitle_data.to_pandas()

In [7]:
dat["de_formality"].value_counts()

0    807766
1    107790
2     80258
3       711
Name: de_formality, dtype: int64